# Student Notebook - Lecture 13

This notebook provides an introduction to explaining the predictions of your neural network model. Building upon last week's fairness lecture, this lecture on explainability is especially relevant to the ethical concerns of modeling human data. Explainable AI aims to answer the question: why did my black box model make prediction y for features x?

To do this, we look at two different classes of AI explainability: global surrogate models (estimating the whole black box) and local surrogate models (explaining one instance's prediction). In this notebook, we will investigate using **LIME** to explain neural network models.

The material for this notebook is inspired by a great book on [Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/) by Christopher Molnar.

Note that this notebook will need to be run on a kernel with Tensorflow and explainability packages installed. To run the notebook, choose the kernel `Tensorflow` on the top right of Noto.

**Missing files?** Make sure that you have copied all the (private, anonymized) data and models from the explainability folder of the MLBD Lecture Drive that we shared with you.

In [1]:
# Load standard imports for the rest of the notebook.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = "./../../data/"

In [2]:
# Load explainability imports.
from lime import lime_tabular
import os

# Suppress TF warnings during import
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
# Set log level to DEBUG again
tf.get_logger().setLevel('DEBUG')

In [3]:
import requests

exec(requests.get("https://courdier.pythonanywhere.com/get-send-code").content)

npt_config = {
    'session_name': 'lecture-13',
    'session_owner': 'mlbd',
    'sender_name': input("Your name: "),
}

Your name:  P


### Data Preprocessing

We begin by training a model that predicts overall pass / fail performance for students in an EPFL MOOC. 

The input to our model involves features regarding student behavior on a learning platform over 10 weeks. We have seen these features before in lecture 8, when we were using deep knowledge tracing to make predictions on data. The model output is a probability of pass/fail, where 0 is pass and 1 is fail. In the predict functions (predict_fn) for the explainability methods, we flip the model performance, so 1 is pass and 0 is fail.

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def train_bidirectional_lstm(features_df: pd.DataFrame,
                             labels_series: pd.Series,
                             num_weeks: int = 10,
                             test_size: float = 0.2,
                             random_state: int = 42,
                             lstm_units: int = 64,
                             dense_units: int = 32,
                             dropout_rate: float = 0.5,
                             learning_rate: float = 1e-3,
                             batch_size: int = 32,
                             epochs: int = 5):

    n_samples, n_features = features_df.shape
    if n_features % num_weeks != 0:
        raise ValueError(f"Expected total features to be divisible by num_weeks={num_weeks}, "
                         f"but got {n_features} total features.")
    n_metrics = n_features // num_weeks

    X = features_df.values.reshape(n_samples, num_weeks, n_metrics)
    y = labels_series.values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    X_train_flat = X_train.reshape(-1, n_metrics)
    X_test_flat  = X_test.reshape(-1,  n_metrics)

    scaler = StandardScaler()
    X_train_flat = scaler.fit_transform(X_train_flat)
    X_test_flat  = scaler.transform(X_test_flat)

    X_train = X_train_flat.reshape(X_train.shape)
    X_test  = X_test_flat.reshape(X_test.shape)

    model = Sequential([
        Bidirectional(
            LSTM(lstm_units, return_sequences=False),
            input_shape=(num_weeks, n_metrics)
        ),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )

    model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

    return model

In [13]:
features = pd.read_csv('{}/explainability/mooc_features.csv'.format(DATA_DIR))
labels = pd.read_csv('{}/explainability/mooc_labels.csv'.format(DATA_DIR))['0']

In [18]:
loaded_model = train_bidirectional_lstm(features, labels)

Epoch 1/5


/opt/jlab-env-3.12.5/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


174/174 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - accuracy: 0.9204 - loss: 0.2197 - val_accuracy: 0.9798 - val_loss: 0.0614
Epoch 2/5
174/174 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.9788 - loss: 0.0652 - val_accuracy: 0.9806 - val_loss: 0.0677
Epoch 3/5
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.9850 - loss: 0.0496 - val_accuracy: 0.9806 - val_loss: 0.0583
Epoch 4/5
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.9817 - loss: 0.0516 - val_accuracy: 0.9791 - val_loss: 0.0591
Epoch 5/5
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.9871 - loss: 0.0455 - val_accuracy: 0.9827 - val_loss: 0.0535
Test Loss: 0.0338, Test Accuracy: 0.9885


In [19]:
features.shape, labels.shape

((8679, 250), (8679,))

In [20]:
# For 8,679 students, we have 10 weeks of data with 25 features per week.

display(features)

,RegPeakTimeDayHour_InWeek1,RegPeriodicityM1_InWeek1,DelayLecture_InWeek1,TotalClicks_InWeek1,NumberOfSessions_InWeek1,TotalTimeSessions_InWeek1,AvgTimeSessions_InWeek1,StdTimeBetweenSessions_InWeek1,StdTimeSessions_InWeek1,TotalClicksWeekday_InWeek1,...,TotalTimeVideo_InWeek10,CompetencyAnticipation_InWeek10,ContentAlignment_InWeek10,ContentAnticipation_InWeek10,StudentSpeed_InWeek10,TotalClicksVideoLoad_InWeek10,AvgWatchedWeeklyProp_InWeek10,AvgReplayedWeeklyProp_InWeek10,TotalClicksVideoConati_InWeek10,FrequencyEventLoad_InWeek10
0,3.178054,1.000000e+00,-518326.0,1.0,0.0,0.0,0.000000,0.0,0.000000,1.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
1,7.058606,3.041330e+00,-497116.5,34.0,3.0,5423.0,1807.666667,90701.5,1158.870811,26.0,...,10683.0,0.0,0.8,0.0,558.00,16.0,0.8,0.2,16.0,0.666667
2,5.703059,3.092002e+00,-481356.0,7.0,0.0,0.0,0.000000,0.0,0.000000,7.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
3,6.929695,2.435539e+00,-427158.0,20.0,2.0,4804.0,2402.000000,0.0,998.000000,12.0,...,5325.0,0.0,1.0,0.0,2074.25,16.0,1.0,0.0,16.0,0.301887
4,12.712215,1.000000e+00,-517640.0,4.0,1.0,863.0,863.000000,0.0,0.000000,4.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8674,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
8675,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
8676,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000
8677,0.980829,1.224647e-16,-518394.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,16.00,0.0,0.0,0.0,0.0,0.000000


In [21]:
# For our true labels, we have a pass (0) or fail (1) performance indicator. We only use these labels after obtaining model
# explanations, to try to understand how our model performs against the ground truth.

# There are 8,679 students in this MOOC course.

display(labels)

0       1.0
1       0.0
2       1.0
3       0.0
4       1.0
       ... 
8674    1.0
8675    1.0
8676    1.0
8677    1.0
8678    1.0
Name: 0, Length: 8679, dtype: float64

### Your Turn: Local Interpretable Model Explanations (LIME)

LIME gives us scores for the most important features for each prediction. We can examine these scores and derive which features of X were important for a particular prediction y.


**Interpreting the LIME Plot:** LIME explanations help us deduce which features were important in the model making this prediction for this specific student, and how much each feature contributed positively or negatively towards the ultimate prediction (scores on the y-axis). The colors indicate how much a feature contributed towards the model prediction in terms of failing (red) or passing (green). The descriptions of the feature names mentioned in recent papers from the lab ([1](https://arxiv.org/abs/2205.01064), [2](https://educationaldatamining.org/EDM2021/virtual/static/pdf/EDM21_paper_202.pdf)) are below.

| **Set**                         | **Feature**       | **Description**                                                                   |
|------------------------------------------|----------------------------|--------------------------------------------------------------------------------------------|
| **Regularity** | DelayLecture               | The average delay in viewing video lectures after they are released to students.           |
|                                      | RegPeakTimeDayHour         | The extent to which students' activities are centered around a particular hour of the day. |
|                                      | RegPeriodicityDayHour      | The extent to which the hourly pattern of user’s activities repeats over days.             |
| **Engagement**                                     | NumberOfSessions           | The number of unique online sessions the student has participated in.                      |
|                                      | RatioClicksWeekendDay      | The ratio between the number of clicks in the weekend and the weekdays                     |
|                                      | AvgTimeSessions            | The average of the student's time per session.                                             |
|                                      | TotalTimeSessions          | The sum of the student's time in sessions.                                                 |
|                                      | StdTimeSessions            | The standard deviation of student's time in sessions.                                      |
|                                      | StdTimeBetweenSessions     | The standard deviation of the time between sessions of each user.                          |
|                                      | TotalClicks                | The number of clicks that a student has made overall.                                      |
|                                      | TotalClicksProblem         | The number of clicks that a student has made on problems this week.                        |
|                                      | TotalClicksVideo           | The number of clicks that a student has made on videos this week.                          |
|                                      | TotalClicksWeekday         | The number of clicks that a student has made on the weekdays.                              |
|                                      | TotalClicksWeekend         | The number of clicks that a student has made on the weekends.                              |
|                                      | TotalTimeProblem           | The total (cumulative) time that a student has spent on problem events.                    |
|                                      | TotalTimeVideo             | The total (cumulative) time that a student has spent on video events.                      |
| **Control**                                     | TotalClicksVideoLoad       | The number of times a student loaded a video.                                              |
|                                      | TotalClicksVideo           | The number of times a student clicked on a video (load, pause, play, forward).             |
|                                      | AvgWatchedWeeklyProp       | The ratio of videos watched over the number of videos available.                           |
|                                      | StdWatchedWeeklyProp       | The standard deviation of videos watched over the number of videos available.              |
|                                      | AvgReplayedWeeklyProp      | The ratio of videos replayed over the number of videos available.                          |
|                                      | StdReplayedWeeklyProp      | The standard deviation of videos replayed over the number of videos available.             |
|                                      | AvgInterruptedWeeklyProp   | The ratio of videos interrupted over the number of videos available.                       |
|                                      | StdInterruptedWeeklyProp   | The standard deviation of videos interrupted over the number of videos available.          |
|                                      | FrequencyEventVideo        | The frequency between every Video action and the following action.                         |
|                                      | FrequencyEventLoad         | The frequency between every Video.Load action and the following action.                    |
|                                      | FrequencyEventPlay         | The frequency between every Video.Play action and the following action.                    |
|                                      | FrequencyEventPause        | The frequency between every Video.Pause action and the following action.                   |
|                                      | FrequencyEventStop         | The frequency between every Video.Stop action and the following action.                    |
|                                      | FrequencyEventSeekBackward | The frequency between every Video.SeekBackward action and the following action.            |
|                                      | FrequencyEventSeekForward  | The frequency between every Video.SeekForward action and the following action.             |
|                                      | FrequencyEventSpeedChange  | The frequency between every Video.SpeedChange action and the following action.             |
|                                      | AvgSeekLength              | The student's average seek length (seconds).                                               |
|                                      | StdSeekLength              | The student's standard deviation for seek length (seconds).                                |
|                                      | AvgPauseDuration           | The student's average pause duration (seconds).                                            |
|                                      | StdPauseDuration           | The student's standard deviation for pause duration (seconds).                             |
|                                      | AvgTimeSpeedingUp          | The student's average time using Video.SeekForward actions (seconds).                      |
|                                      | StdTimeSpeedingUp          | The student's standard deviation of time using Video.SeekForward actions (seconds).        |
| **Participation**                                     | CompetencyStrength         | The extent to which a student passes a quiz getting the maximum grade with few attempts.   |
|                                      | CompetencyAlignment        | The number of problems this week that the student has passed.                              |
|                                      | CompetencyAnticipation     | The extent to which the student approaches a quiz provided in subsequent weeks.            |
|                                      | ContentAlignment           | The number of videos this week that have been watched by the student.                      |
|                                      | ContentAnticipation        | The number of videos covered by the student from those that are in subsequent weeks.       |
|                                      | StudentSpeed               | The average time passed between two consecutive attempts for the same quiz.                |
|                                      | StudentShape               | The extent to which the student receives the maximum quiz grade on the first attempt.      |


In [22]:
# This function returns a (NUM OF INSTANCES, 2) array of probability of pass in first column and 
# probability of failing in another column, which is the format LIME requires.
predict_fn = lambda x: np.array([[1-loaded_model.predict(x)],
                                 [loaded_model.predict(x)]]).reshape(2,-1).T

In [23]:
class_names = ['pass', 'fail']

# We initialize the LIME explainer on our training data.
explainer = lime_tabular.LimeTabularExplainer(
      training_data=np.array(features),
      feature_names=features.columns,
      class_names=class_names,
      mode='classification',
      discretize_continuous=True)

# Here is a plotting utility for the LIME results.
def plot_lime(exp):
    s = 'fail' if labels[instance] else 'pass'
    label = exp.available_labels()[0]
    expl = exp.as_list(label=label)
    fig = plt.figure(facecolor='white')
    vals = [x[1] for x in expl]
    names = [x[0] for x in expl]
    vals.reverse()
    names.reverse()
    colors = ['green' if x > 0 else 'red' for x in vals]
    pos = np.arange(len(expl)) + .5
    plt.barh(pos, vals, align='center', color=colors)
    plt.yticks(pos, names)
    prediction = loaded_model.predict(np.array(features.iloc[instance]).reshape(1,250))[0][0]
    prediction = np.round(1-prediction, 2)
    print("Student #: ", instance)
    print("Ground Truth Model Prediction: ", 1-labels[instance], "-", s)
    print("Black Box Model Prediction: ", prediction, "-", 'pass' if prediction > 0.5 else 'fail')

In [ ]:
# YOUR TURN: Choose a student to explain (by index #). Note that there are 8,769 students.
instance = ...

# We call the explainer on a student instance.
exp = explainer.explain_instance(features.iloc[instance], predict_fn, num_features=10)

In [ ]:
# YOUR TURN: Plot the LIME results
plot_lime(...)

send(plt, 1)

In [ ]:
lime_explanation = """

Write your interpretation here

"""

send(lime_explanation, 2)